In [32]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import datetime as dt
today_date = dt.date.today().strftime('%Y-%m-%d')

query_list = [
            (f"(TheCircle (#TheCircle) lang:en  since:2022-05-01 until:{today_date})"),
            (f"(theCircle (#TheCircleNetflix) lang:en  since:2022-05-01 until:{today_date})"),
            (f"(TheCircle (#TheCircleNetflix) lang:en  since:2022-05-01 until:{today_date})"),
            (f"(thecircle (#TheCircle) lang:en  since:2022-05-01 until:{today_date})"),
            (f"(the circle (#TheCircle) lang:en  since:2022-05-01 until:{today_date})"),
            (f"(the circle (#TheCircleNetflix) lang:en  since:2022-05-01 until:{today_date})") ,
            (f"(the circle (#TheCircle4) lang:en  since:2022-05-01 until:{today_date})") ,
                     
]
tweets = []
limit = 50000

for query in query_list:
    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
        if len(tweets) > limit:
            break
        else:
            tweets.append([tweet.date, tweet.user.username, tweet.content])
            
df = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet'])
# to save to csv
df.to_csv('tweet_combo_all.csv')

In [34]:
import pandas as pd
twt =pd.read_csv('tweet_combo_all.csv')
twt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22019 entries, 0 to 22018
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  22019 non-null  int64 
 1   Date        22019 non-null  object
 2   User        22019 non-null  object
 3   Tweet       22019 non-null  object
dtypes: int64(1), object(3)
memory usage: 688.2+ KB


In [35]:
twt.head()

,Unnamed: 0,Date,User,Tweet
0,0,2022-05-13 23:57:47+00:00,beardsational,"I’m sorry, but if you received two party invit..."
1,1,2022-05-13 23:57:14+00:00,KingDelfrano,enjoying this season of #TheCircle so far… nob...
2,2,2022-05-13 23:51:51+00:00,KMJeezy,Ok I aint like Yu Ling at first but she is com...
3,3,2022-05-13 23:49:00+00:00,Blake_Jorgensen,I need someone to watch #TheCircle with me so ...
4,4,2022-05-13 23:47:58+00:00,jceron12,I haven’t been seeing any tweets about #thecir...


In [36]:
df = twt